# API Estimator

API Estimator es otro API de alto nivel en TensorFlow que simplifica en gran medida la programación.

Los estimadores encapsulan:

* Definición de modelos

```
estimator = tf.estimator.LinearRegressor(    
        feature_columns=feature_cols ...)	
```

* Training

```
train(input_fn, hooks=None, steps=None, max_steps=None, saving_listeners=None) 	
```         
Donde input_fn es una función que provee de los datos de la forma (características, etiquetas) respecto los datos de training

* Evaluación
```
evaluate(input_fn, steps=None, hooks=None, checkpoint_path=None, name=None) 
```

Donde input_fn es una función que provee de los datos de la forma (características, etiquetas) respecto a los datos de test

* Predicción

```
predict(input_fn, predict_keys=None, hooks=None, checkpoint_path=None, yield_single_examples=True)
```

Donde input_fn es una función que construye los datos de la forma (características, etiquetas) 
Predict_keys es la lista de claves a predecir




## ¿Cómo los usamos?:
    
#### * Funciones para importar dataset
  
  De cada dataset debemos obtener dos objetos:
  * Un diccionario en el cual las claves son características y valores de los tensores que contienen los datos de esos atributos
  * Un tensor que contiene las etiquetas/labels
  
  Ejemplo de cómo debe estructurarse
    ```
    def input_fn(dataset):
    ...  # transformar el dataset, obtener el diccionario y las características
    return feature_dict, label
    ```  
    
#### * Definición de atributos característicos
   Cada tf.feature_column identifica el nombre de una de nuestras características/atributos, las columnas de nuestro dataset
    
    ```
    altura = tf.feature_column.numeric_column('altura')
    peso = tf.feature_column.numeric_column('peso')
    nota_media = tf.feature_column.numeric_column('nota_media', normalizer_fn=lambda x: x - nota_global)
    ```

#### * Instanciar el estimador prefabricado
   Veamos un ejemplo con el estimador LinearClassifier
   
    ```
    estimator = tf.estimator.LinearClassifier(
        feature_columns=[altura, peso, nota_media])
     ```
    
#### * Llamar al método de training, evaluate o predict
   Definimos para por ejemplo el training la entrada a la función de importar datos
   
   ```   
   estimator.train(input_fn=my_training_set, steps=2000)
   ```


In [12]:
#Ejemplo de una regresión lineal
import pandas as pd
import tensorflow as tf

train_data = pd.read_csv("/home/user-es-ubuntu/Escritorio/data/iris_train.csv", names=['f1', 'f2', 'f3', 'f4', 'f5'])
test_data = pd.read_csv("/home/user-es-ubuntu/Escritorio/data/iris_test.csv", names=['f1', 'f2', 'f3', 'f4', 'f5'])

train_x = train_data[['f1', 'f2', 'f3','f4']]
train_y = train_data.ix[:, 'f5']

test_x = test_data[['f1', 'f2', 'f3', 'f4']]
test_y = test_data.ix[:, 'f5']

train_data.dtypes

feature_columns = [tf.feature_column.numeric_column(key=key) for key in train_x.keys()]

classifier = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=3)

print("creado estimador")

def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    return dataset.shuffle(1000).repeat().batch(batch_size)

classifier.train(
    input_fn=lambda: train_input_fn(train_x, train_y, 50),
    steps=2000)

def eval_input_fn(features, labels, batch_size):
    features = dict(features)
    inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    return dataset.batch(batch_size)

eval_result = classifier.evaluate(
    input_fn=lambda: eval_input_fn(test_x, test_y, 100))

print('\Accuracy del modelo: {:0.3f}\n'.format(**eval_result))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
W0829 18:39:24.505276 139816252819264 estimator.py:1811] Using temporary folder as model directory: /tmp/tmp_tb70roi


creado estimador

Test set accuracy: 0.867

